In [33]:
!pip install mercantile

  Obtaining dependency information for mercantile from https://files.pythonhosted.org/packages/b2/d6/de0cc74f8d36976aeca0dd2e9cbf711882ff8e177495115fd82459afdc4d/mercantile-1.2.1-py3-none-any.whl.metadata
Using cached mercantile-1.2.1-py3-none-any.whl (14 kB)


In [34]:
import os
import requests
import mercantile
from shapely.geometry import shape, Polygon
from concurrent.futures import ThreadPoolExecutor, as_completed
from time import sleep

In [35]:


# Helper function to download a single tile with retries
def download_tile(xyz_url, z, x, y, xyz_dir, retries=3, delay=1):
    tile_url = xyz_url.format(z=z, x=x, y=y)
    tile_path = os.path.join(xyz_dir, str(z), str(x), f"{y}.png")
    
    # Create directories if they do not exist
    os.makedirs(os.path.dirname(tile_path), exist_ok=True)

    # Check if tile already exists
    if os.path.exists(tile_path):
        return  # Tile already cached
    
    # Try to download the tile with retries
    attempt = 0
    while attempt < retries:
        try:
            response = requests.get(tile_url, timeout=10)  # Set a timeout for requests
            if response.status_code == 200:
                with open(tile_path, 'wb') as f:
                    f.write(response.content)
                return  # Successfully downloaded
            else:
                print(f"Failed to download tile {z}/{x}/{y}, status: {response.status_code}")
                break  # Don't retry if server gives a bad response
        except Exception as e:
            print(f"Error downloading tile {z}/{x}/{y}: {e}")
            attempt += 1
            if attempt < retries:
                print(f"Retrying tile {z}/{x}/{y}... (attempt {attempt}/{retries})")
                sleep(delay)  # Wait before retrying
    print(f"Failed to download tile {z}/{x}/{y} after {retries} attempts")

# Main function to download and cache XYZ tiles
def cache_xyz_tiles(xyz_url, xyz_dir, zoom, aoi=None, max_workers=10, verbose=True):
    """
    Downloads XYZ tiles and caches them locally.

    Args:
    - xyz_url (str): The URL template for the XYZ tile service.
    - xyz_dir (str): Directory to store the cached tiles.
    - zoom (int or range): The zoom level(s) for which to download tiles.
    - aoi (dict, optional): A GeoJSON object defining the Area of Interest (AOI). Defaults to global.
    - max_workers (int, optional): Number of threads for concurrent downloads. Default is 10.
    - verbose (bool, optional): If True, prints progress information. Default is True.
    """
    
    # If no AOI is provided, use the entire world extent
    if aoi is None:
        # Global extent (Web Mercator bounds)
        aoi_extent = (-180, -85.0511, 180, 85.0511)  # As a tuple (min_lon, min_lat, max_lon, max_lat)
    else:
        # Extract bounding box from AOI GeoJSON
        polygon = shape(aoi['features'][0]['geometry'])
        if isinstance(polygon, Polygon):
            aoi_extent = polygon.bounds  # This will return (min_lon, min_lat, max_lon, max_lat)
        else:
            raise ValueError("AOI geometry must be a Polygon")

    min_lon, min_lat, max_lon, max_lat = aoi_extent
    
    # Ensure zoom is iterable (even if it's just one level)
    if isinstance(zoom, int):
        zoom_levels = [zoom]
    else:
        zoom_levels = range(zoom[0], zoom[1] + 1)
    
    # Use mercantile to calculate the list of tiles
    total_tiles = 0
    for z in zoom_levels:
        tile_list = list(mercantile.tiles(min_lon, min_lat, max_lon, max_lat, zooms=[z]))
        total_tiles += len(tile_list)
        if verbose:
            print(f"Zoom level {z}: {len(tile_list)} tiles to download.")

        # Download the tiles using multithreading
        futures = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for tile in tile_list:
                futures.append(executor.submit(download_tile, xyz_url, tile.z, tile.x, tile.y, xyz_dir))

            # Track progress and handle exceptions
            tile_count = 0
            for future in as_completed(futures):
                try:
                    future.result()  # Raise any exceptions caught by threads
                except Exception as e:
                    print(f"Error in future: {e}")

                tile_count += 1
                if verbose:
                    print(f"Progress: {tile_count}/{len(tile_list)} tiles downloaded for zoom {z}", end="\r")
    
    print("\nTile caching complete.")




# Example usage:

In [39]:
# Example usage:
# Use the global default extent without providing AOI
xyz_url = "https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps/cca86b15cd17eebfec8b381ec7166df4-f607df06b0198e13a0b51c3376cff8c1/tiles/{z}/{x}/{y}"
xyz_dir = "/Users/maples/GitHub/EarthEngine_2_Globe/caches"
zoom = 4  # Download tiles for zoom level 4 globally

# Call the function without an AOI
cache_xyz_tiles(xyz_url, xyz_dir, zoom)

Zoom level 4: 256 tiles to download.
Error downloading tile 4/2/5: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Read timed out. (read timeout=10)
Retrying tile 4/2/5... (attempt 1/3)
Error downloading tile 4/2/6: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Read timed out. (read timeout=10)
Retrying tile 4/2/6... (attempt 1/3)
Error downloading tile 4/3/5: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Read timed out. (read timeout=10)
Retrying tile 4/3/5... (attempt 1/3)
Error downloading tile 4/3/6: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Read timed out. (read timeout=10)
Retrying tile 4/3/6... (attempt 1/3)
Error downloading tile 4/4/5: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Read timed out. (read timeout=10)
Retrying tile 4/4/5... (attempt 1/3)
Error downloading tile 4/4/6: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Read timed out. (read timeout=10